本文内容主要来自于：
- https://arxiv.org/pdf/1502.03167.pdf
- http://blog.csdn.net/hjimce/article/details/50866313

## 摘要
训练深度神经元网络的一个难点在于：在训练的过程中，跟随这上一层参数的改变，每层的输入分布在不停的改变。(the dictribution of each layer's input changes durign training, as the parameters of the previous layers change.) 这使得我们需要相对比较小的学习率，以及比较好的参数初始化，来达到训练的目的。而这将会大大的降低训练的速度。我们称这种现象为internal covariate shift，并尝试通过规范化每个层的输入来化解这个问题。我们的方法把normalization作为神经网络的一部分，并对每个batch的内容进行规范化。<br>
batch normalization使得我们可以使用比较大的学习率，并且不用那么在乎参数的初始化。另外，他有时也相当于一个调节者（regularizer），在某些情况下，他可以实现dropout的功能。

## 介绍
我们之所以认为每层输入的分布的改变为一个麻烦的问题在于，在这种情况下，每个层的参数需要不断的调整其改变的方向。当一个学习系统的输入分布发生改变，我们称之为experience covariate shift(Shimodaira, 2000)。这种状况可以通过domain adaption(Jiang, 2008)进行处理。然而这种covariate shift可以被从整体细节到其中的一个部分。也就是说一些有利于训练整个学习系统的因素，应用于单个层也是适用的。比如保持训练集和测试集的分布相同(???)。因此，保持输入的分布不变是有利于训练的？？？<br>
保持子网络的输入分布不变同样也有利于该子网络之外的网络。回想一下梯度消失的原因。如果我们可以在训练的过程中维持输入的分布比较稳定，这可以降低优化时进入饱和区域的可能性，从而可以加速训练。<br>
这里我们提出一种方法，Batch normalization，来处理internal covariate shift问题。其主要思想就是调整每个层输入的mean和variance。通过减少梯度对参数标量和初始值的影响，BN同时也有利于整个网络的梯度流。(Batch Normalization also has beneficial effect on the gradient flow through the network, by reduing the dependence of gradients on the scale of the parameters or of their initial values.) Fianlly, Batch Normalization makes it possible to use saturating nonlinearities by preventing the network from getting stuck in the saturated mode.

## 预处理操作选择
说到神经网络输入数据预处理，最好的算法莫过于白化预处理。然而白化计算量太大了，很不划算，还有就是白化不是处处可微的，所以在深度学习中，其实很少用到白化。经过白化预处理后，数据满足条件：a、特征之间的相关性降低，这个就相当于pca；b、数据均值、标准差归一化，也就是使得每一维特征均值为0，标准差为1。如果数据特征维数比较大，要进行PCA，也就是实现白化的第1个要求，是需要计算特征向量，计算量非常大，于是为了简化计算，作者忽略了第1个要求，仅仅使用了下面的公式进行预处理，也就是近似白化预处理：$x' = \frac{x - M}{S}$

## BN算法实现
1、BN算法概述
经过前面简单介绍，这个时候可能我们会想当然的以为：好像很简单的样子，不就是在网络中间层数据做一个归一化处理嘛，这么简单的想法，为什么之前没人用呢？然而其实实现起来并不是那么简单的。其实如果是仅仅使用上面的归一化公式，对网络某一层A的输出数据做归一化，然后送入网络下一层B，这样是会影响到本层网络A所学习到的特征的。打个比方，比如我网络中间某一层学习到特征数据本身就分布在S型激活函数的两侧，你强制把它给我归一化处理、标准差也限制在了1，把数据变换成分布于s函数的中间部分，这样就相当于我这一层网络所学习到的特征分布被你搞坏了，这可怎么办？于是文献使出了一招惊天地泣鬼神的招式：变换重构，引入了可学习参数γ、β，这就是算法关键之处：$y^{(k)}=\gamma^{(k)}\hat{x}^{(k)}+\beta^{(k)}$<br>
每一个神经元xk都会有一对这样的参数γ、β。这样其实当：$\gamma = S, \beta = M$. 是可以恢复出原始的某一层所学到的特征的。因此我们引入了这个可学习重构参数γ、β，让我们的网络可以学习恢复出原始网络所要学习的特征分布。最后Batch Normalization网络层的前向传导过程公式就是：
![BN](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/BN.png)

根据文献说，BN可以应用于一个神经网络的任何神经元上。文献主要是把BN变换，置于网络激活函数层的前面。在没有采用BN的时候，激活函数层是这样的：
z=g(Wu+b)<br>
也就是我们希望一个激活函数，比如s型函数s(x)的自变量x是经过BN处理后的结果。因此前向传导的计算公式就应该是：
z=g(BN(Wu+b))<br>
其实因为偏置参数b经过BN层后其实是没有用的，最后也会被均值归一化，当然BN层后面还有个β参数作为偏置项，所以b这个参数就可以不用了。因此最后把BN层+激活函数层就变成了：
z=g(BN(Wu))<br>

## Batch Normalization在CNN中的使用
通过上面的学习，我们知道BN层是对于每个神经元做归一化处理，甚至只需要对某一个神经元进行归一化，而不是对一整层网络的神经元进行归一化。既然BN是对单个神经元的运算，那么在CNN中卷积层上要怎么搞？假如某一层卷积层有6个特征图，每个特征图的大小是100*100，这样就相当于这一层网络有6*100*100个神经元，如果采用BN，就会有6*100*100个参数γ、β，这样岂不是太恐怖了。因此卷积层上的BN使用，其实也是使用了类似权值共享的策略，把一整张特征图当做一个神经元进行处理。<br>
卷积神经网络经过卷积后得到的是一系列的特征图，如果min-batch sizes为m，那么网络某一层输入数据可以表示为四维矩阵(m,f,p,q)，m为min-batch sizes，f为特征图个数，p、q分别为特征图的宽高。在cnn中我们可以把每个特征图看成是一个特征处理（一个神经元），因此在使用Batch Normalization，mini-batch size 的大小就是：m*p*q，于是对于每个特征图都只有一对可学习参数：γ、β。说白了吧，这就是相当于求取所有样本所对应的一个特征图的所有神经元的平均值、方差，然后对这个特征图神经元做归一化。